# Import packages

In [ ]:
!pip install us
!pip install timedelta
import us
import pickle
import pandas as pd
import plotly.express as px
from google.colab import drive
from datetime import datetime
import copy
import timedelta

drive.mount('/content/drive')
root_path = '<path to the root path'

Mounted at /content/drive


# Time difference function

In [ ]:
def time_difference(T1, T2):
  # Converting the dates to the datetime objects
  T1 = datetime.strptime(T1, '%m/%d/%Y')
  T2 = datetime.strptime(T2, '%m/%d/%Y')
  # Computing the delta time
  td = timedelta.Timedelta(T2 - T1)

  return td

# Loading the data

In [ ]:
payments_df = pd.read_csv(f'{root_path}Payments.csv')
charges_df = pd.read_csv(f'{root_path}Charges.csv')
adjustments_df = pd.read_csv(f'{root_path}Adjustments.csv')
demographic_data_df = pd.read_csv(f'{root_path}Patient Demographic Data.csv')

# Top 10 popular procedures

Top 10 procedures that was performed the most.

In [ ]:
# Get the number of the requests for each procedure
Procedures_count = charges_df['Name'].value_counts()

#Create a bar plot
fig = px.bar(x=Procedures_count.index[:10], y=Procedures_count.values[:10], title='Top 10 popular procedures', labels={'x': 'Procedure name', 'y': '#Requests'},  text_auto='.2s')

# Show the plot
fig.show()


#Top 10 most expensive procedures

In the case that we want to find the most expensive procedures, we have to take an average cost for each procedure.

In [ ]:
# Computes the cost on average for each procedure to be done
Procedure_cost = charges_df[['Name', 'Amount']].groupby(by = 'Name').mean().reset_index().sort_values('Amount', ascending = False).head(10)

#Create a bar plot
fig = px.bar(x=Procedure_cost['Name'], y=Procedure_cost['Amount'], title='Top 10 most expensive procedures', labels={'x': 'Procedure name', 'y': 'Cost'},  text_auto='.2s')


# Show the plot
fig.show()



# Top 10 most credited procedures

We show the top 10 procedures that credited the highest total money

In [ ]:
# Computes the cost on average for each procedure to be done
Procedure_credited = charges_df[['Name', 'Amount']].groupby(by = 'Name').sum().reset_index().sort_values('Amount', ascending = False).head(10)

#Create a bar plot
fig = px.bar(x=Procedure_credited['Name'], y=Procedure_credited['Amount'], title='Top 10 most credited procedures', labels={'x': 'Procedure name', 'y': 'Total credit'},  text_auto='.2s')

# Show the plot
fig.show()

# Doctor's most performed procedures

In [ ]:
# This dictionary contains the info for each doctor
Doctors_procedures = []

for doctor_id in charges_df['Doctor'].unique():

  # filtering the records for each specific doctor
  filtered_rec = charges_df[charges_df['Doctor']== doctor_id]

  # Procedures distribution
  Procedure_dist = filtered_rec['Name'].value_counts()

  Doctors_procedures.append({'Doctor': doctor_id,
                             'Most procedure performed ' : Procedure_dist.index[0],
                             'Total count' : Procedure_dist.values[0]})

# Forming the dataframe for the doctors to represent which procedure they performed the most
pd.DataFrame(Doctors_procedures).sort_values('Total count', ascending = False)

,Doctor,Most procedure performed,Total count
0,5,Estradiol,4224
4,9,Office Visit Established Patient Level 4,3649
5,6,Storage Egg Monthly,3077
1,3,Office Visit Established Patient Level 4,1935
6,0,Storage Embryo Monthly Yrs 1-2,1729
10,1,Office Visit Established Patient Level 4,1525
2,8,Storage Embryo Monthly Yrs 3-5,1415
3,7,Office Visit Established Patient Level 4,843
9,2,Storage Embryo Monthly Yrs 1-2,807
7,4,Storage Embryo Monthly Yrs 3-5,269


# Doctor's Amount credited

In [ ]:
# Computes the cost on average for each procedure to be done
Doctors_amount_credited = charges_df[['Doctor', 'Amount']].groupby(by = 'Doctor').sum().reset_index().sort_values('Amount', ascending = False)

#Create a bar plot
fig = px.bar(x=Doctors_amount_credited['Doctor'].astype(str), y=Doctors_amount_credited['Amount'], title='Doctors` amount credited', labels={'x': 'Doctor', 'y': 'Total credit'},  text_auto='.2s')

# Show the plot
fig.show()

In [ ]:
Doctors_amount_credited

,Doctor,Amount
5,5,9195326.88
9,9,5809286.44
6,6,4413143.36
3,3,4138059.86
0,0,2621673.13
8,8,2501345.81
1,1,2277034.69
7,7,2200450.04
2,2,1766848.61
4,4,214538.89


# Doctor's different procedures

In this plot, we are going to show how many different procedures each doctor can perform.

In [ ]:
# This dictionary contains the info for each doctor
Doctors_procedures_distinct = []

for doctor_id in charges_df['Doctor'].unique():

  # filtering the records for each specific doctor
  filtered_rec = charges_df[charges_df['Doctor']== doctor_id]

  Doctors_procedures_distinct.append({'Doctor': doctor_id,
                                      'Procedures num' : len(filtered_rec['Name'].unique())})

# Forming the dataframe for the doctors to represent how many different procedures they can perform
Doctors_procedures_df = pd.DataFrame(Doctors_procedures_distinct).sort_values('Procedures num', ascending = False)

#Create a bar plot
fig = px.bar(x=Doctors_procedures_df['Doctor'].astype(str), y=Doctors_procedures_df['Procedures num'], title='Doctors` procedures performed', labels={'x': 'Doctor', 'y': '#Procedures capable of'},  text_auto='.3s')

# Show the plot
fig.show()

# Demographic insights

## Top 5 states

We are going to identify the top 5 states that most of the patients are coming from.

In [ ]:
# There were some null values in the column
# represented by 0 -> exclude them from the analysis
States_withoutNull = demographic_data_df[demographic_data_df['State'] != '0']

# Get the number of the patients from each state
State_counts = States_withoutNull['State'].value_counts()

#Create a bar plot
fig = px.bar(x=State_counts.index[:5], y=State_counts.values[:5], title='Top 5 states - #Patients', labels={'x': 'State', 'y': '#Patients'},  text_auto='.2s')

# Show the plot
fig.show()

# Top 10 international countries

In this section we are going to identify the top 10 countries that had the highest number of patients

In [ ]:
# Getting the name of the states
state_names = [state.abbr for state in us.states.STATES_AND_TERRITORIES]

# Only keep the records that their state is not in the US
International = States_withoutNull[~(States_withoutNull['State'].isin(state_names))]

# Get the number of the patients from each state
International_count = International['State'].value_counts()

#Create a bar plot
fig = px.bar(x=International_count.index[:10], y=International_count.values[:10], title='Top 10 foreign countries - #Patients', labels={'x': 'Country name', 'y': '#Patients'},  text_auto='.2s')

# Show the plot
fig.show()

In [ ]:
# MX - Mexico
# AE - United Arab Emirates

# In debt insights

Sort the data based on the encounter date

In [ ]:
# drop the records from the charges that they have the encounter
# as null value
Charges_df_copy = charges_df.copy().dropna(subset = 'Encounter')

# Add a new column for the unix time of the encounter date
Charges_df_copy['Unix_encounter'] = Charges_df_copy.apply(lambda x: datetime.strptime(str(x['Encounter']), '%m/%d/%Y').timestamp(), axis = 1)

# Sorting the charges' records based on the unix timestamp
Charges_df_copy.sort_values(by = 'Unix_encounter', inplace = True)

Charges_df_copy

,Unique Acct #,Posted,Encounter,Code,Name,Doctor,Facility,Department,Billed To,Amount,Unix_encounter
14952,100018,3/17/2022,1/1/2000,99211,Office Visit Established Patient Level 1,5,2,Evaluation And Mgmt,Zero,0.00,9.466848e+08
137420,106033,8/22/2023,1/3/2018,S4027Y6,Storage Embryo Monthly Over 5 Years,5,2,Cryo Storage,Patient,1500.00,1.514938e+09
79424,120163,12/29/2022,1/15/2018,89255,Preparation Of Embryo Transfer,5,2,Embryology,Patient,20.00,1.515974e+09
79405,120163,12/29/2022,1/15/2018,89352,Thawing Of Cryopreserved Embryos,5,2,Embryology,Patient,25.00,1.515974e+09
79402,120163,12/29/2022,5/9/2018,84144,Progesterone,5,2,Endocrinology,Patient,1.22,1.525824e+09
...,...,...,...,...,...,...,...,...,...,...,...
174072,102205,12/31/2023,12/31/2023,99214-25,Office Visit Established Patient Level 4,3,2,Evaluation And Mgmt,Insurance,105.00,1.703981e+09
174074,103032,12/31/2023,12/31/2023,76830,Ultrasound Transvaginal,3,2,Sonography,Insurance,185.00,1.703981e+09
174080,103228,12/31/2023,12/31/2023,99211,Office Visit Established Patient Level 1,2,2,Evaluation And Mgmt,Zero,0.00,1.703981e+09
173671,104095,12/27/2023,1/5/2024,89346,Storage Egg Yearly,5,2,Cryo Storage,Insurance,1200.00,1.704413e+09


## __Saving the customers` data__ (Warning!)

Run the cell below only if you want to update the .pickle file containing the customers` data.

In [ ]:
'''
# Consider a dictionary for each customer
# For each customer we want to track the followings:
# 1- Balance
# 2- The code and the name of the last procedure
# 3- The doctor of the customer that did the last procedure
# 4- The time that the balance got negative - date
# 5- List of the period that the balance was negative and got positive

# Template for each customer
Info_template = {'Balance' : 0,
                 'Last_procedure_code' : None,
                 'Last_procedure_name' : None,
                 'Last_procedure_doctor' : None,
                 'Negative_balance_date': None,
                 'Negative_balance_periods' : []}

# Customers' info
# Each key -> customer id
# Each value -> info template
All_customers_info = {}

# Looping over the records
for record_index in range(Charges_df_copy.shape[0]):


  # Take the customer index
  customer_idx = Charges_df_copy.iloc[record_index]['Unique Acct #']

  # If the customer has not been seen up to now
  if All_customers_info.get(customer_idx) == None:

    # Put a template for that customer
    All_customers_info[customer_idx] = copy.deepcopy(Info_template)

  # Finding the records corresponding to this procedure
  # in the payments -> summing up those payments and updating the customer balance
  procedure_encounter = Charges_df_copy.iloc[record_index]['Encounter']
  procedure_code = Charges_df_copy.iloc[record_index]['Code']
  procedure_charge = Charges_df_copy.iloc[record_index]['Amount']
  procedure_date = Charges_df_copy.iloc[record_index]['Encounter']

  procedure_payments = payments_df[(payments_df['Unique Acct #'] == customer_idx) &
                                   (payments_df['Encounter'] == procedure_encounter) &
                                   (payments_df['Code'] == procedure_code)]

  # Update the balance of the customer
  All_customers_info[customer_idx]['Balance'] += procedure_payments['Amount'].sum()

  # in the adjustments -> summing up those adjustments and updating the customer balance
  procedure_adjustments = adjustments_df[(adjustments_df['Unique Acct #'] == customer_idx) &
                                   (adjustments_df['Encounter'] == procedure_encounter) &
                                   (adjustments_df['Code'] == procedure_code)]
  # Update the balance of the customer
  All_customers_info[customer_idx]['Balance'] += procedure_adjustments['Amount'].sum()

  # deduct the charge from the balance of the customer
  All_customers_info[customer_idx]['Balance'] -= procedure_charge

  # rounding to two decimal points
  All_customers_info[customer_idx]['Balance'] = round(All_customers_info[customer_idx]['Balance'], 2)

  # When the balance gets negative and this procedure made the balance to get negative
  # take into account this date and the other information about the user.
  if All_customers_info[customer_idx]['Balance'] < 0 and  All_customers_info[customer_idx]['Negative_balance_date'] == None:

    # at this point the balance got negative
    All_customers_info[customer_idx]['Negative_balance_date'] = procedure_date

    # Update the name of the last procedure
    All_customers_info[customer_idx]['Last_procedure_name'] = Charges_df_copy.iloc[record_index]['Name']

    # Update the code of the last procedure
    All_customers_info[customer_idx]['Last_procedure_code'] = Charges_df_copy.iloc[record_index]['Code']

    # Update the doctor of the last procedure
    All_customers_info[customer_idx]['Last_procedure_doctor'] = Charges_df_copy.iloc[record_index]['Doctor']

  # If the balance of the customer got positive and previously it was negative, store the days that has been passed in order to
  # have the balance again positive and store the corresponding results to the list of the negative periods
  # 1- The number of days that the balance remained negative
  # 2- The last procedure that made the balance to be negative
  # 3- The Doctor of the last procedure
  # 4- The name of the procedure
  elif (All_customers_info[customer_idx]['Balance'] >= 0) and (All_customers_info[customer_idx]['Negative_balance_date'] != None):

    # Check the days passed from the last negative balance up to this timestamp
    # and add it to the list of periods
    Negative_date = All_customers_info[customer_idx]['Negative_balance_date']

    # Get the date that the balance got negative
    T1 = datetime.strptime(Negative_date, '%m/%d/%Y')

    # Get the date of the procedure
    T2 = datetime.strptime(procedure_date, '%m/%d/%Y')

    # Check the number of the days in this period
    td = timedelta.Timedelta(T2 - T1)

    # If in the same day the balance gets negative and then positive again,
    # don't consider that
    if td.days != 0:

      # adding the period to the list
      All_customers_info[customer_idx]['Negative_balance_periods'].append((td.days, All_customers_info[customer_idx]['Last_procedure_name'],
                                                                            All_customers_info[customer_idx]['Last_procedure_code'],
                                                                           All_customers_info[customer_idx]['Last_procedure_doctor']))

    # Reset the values that we have been considered to do our analysis
    All_customers_info[customer_idx]['Negative_balance_date'] = None
    All_customers_info[customer_idx]['Last_procedure_name'] = None
    All_customers_info[customer_idx]['Last_procedure_code'] = None
    All_customers_info[customer_idx]['Last_procedure_doctor'] = None

# Open the file in binary mode
with open(f'{root_path}Customer_agg_data.pickle', 'wb') as file:

    # Serialize and write the variable to the file
    pickle.dump(All_customers_info, file)
'''
print('Not running the feature extraction step!!!')

## __Loading the customers` data__ (Warning)

Run the cell below only if you want to load the data from the .pickle file which contains the customers` data.

In [ ]:
## Open the file in binary mode
with open(f'{root_path}Customer_agg_data.pickle', 'rb') as file:

    # Deserialize and retrieve the variable from the file
    All_customers_info = pickle.load(file)

Taking a look at some of the records

In [ ]:
# Forming dataframe from the customers aggregated data
Customers_agg_df = pd.DataFrame(All_customers_info).T

# Resetting the indexes in the dataframe
Customers_agg_df.reset_index(inplace = True)

# Renaming the columns name
Customers_agg_df.columns = ['Customer id'] + list(Customers_agg_df.columns)[1:]

# Saving the data about all of the customers
Customers_agg_df.to_csv(f'{root_path}All_customers.csv', index = False)

Customers_agg_df


,Customer id,Balance,Last_procedure_code,Last_procedure_name,Last_procedure_doctor,Negative_balance_date,Negative_balance_periods
0,100018,0.0,None,None,None,None,[]
1,106033,1020.0,None,None,None,None,[]
2,120163,-125.0,S4027Y6,Storage Embryo Monthly Over 5 Years,5,12/22/2023,[]
3,101400,73530.54,None,None,None,None,[]
4,106071,16318.57,None,None,None,None,[]
...,...,...,...,...,...,...,...
5272,103593,-75.0,89343Q,Storage Sperm Quarterly,0,12/28/2023,[]
5273,104166,0.0,None,None,None,None,[]
5274,104232,-250.0,99205,New Patient Office Or Other Outpatient Visit L...,1,12/28/2023,[]
5275,103935,-166.0,89320,Semen Analysis Comprehensive,5,12/28/2023,[]


Taking a look at the customers that had negative periods

In [ ]:
# Calculate how many negative periods each customer had
Customers_agg_df['#Negative_periods'] = Customers_agg_df.apply(lambda x: len(x['Negative_balance_periods']), axis = 1)

# Filter out the customers data that they had negative periods
Customers_agg_df[Customers_agg_df['#Negative_periods'] != 0 ]


,Customer id,Balance,Last_procedure_code,Last_procedure_name,Last_procedure_doctor,Negative_balance_date,Negative_balance_periods,#Negative_periods
134,128098,-264.76,58970,Egg Retrieval,2,1/14/2022,"[(65, Extended Culture, 89272, 0)]",1
152,128442,1537.0,None,None,None,None,"[(7, Progesterone, 84144, 0)]",1
170,100142,47866.01,None,None,None,None,"[(1, Estradiol, 82670, 5), (2, Office Visit Es...",3
225,124195,15413.95,None,None,None,None,"[(11, Estradiol, 82670, 3), (5, Egg Retrieval,...",2
357,121994,-1401.3,A4649,Surgical Supply Miscellaneous,8,6/28/2023,"[(2, Storage Embryo Yearly, 89342, 5), (7, Ult...",3
...,...,...,...,...,...,...,...,...
5001,103879,75.0,None,None,None,None,"[(2, Ultrasound Pelvic Limited, 76857, 6)]",1
5097,123385,-7411.0,89251,CoCulture Of Egg Or Embryo,6,11/27/2023,"[(17, New Patient Office Or Other Outpatient V...",1
5108,120968,39.9,None,None,None,None,"[(15, New Patient Office Or Other Outpatient V...",1
5127,103930,-12706.0,89250,Culture Of Egg Or Embryo,7,12/6/2023,"[(9, Ultrasound Transvaginal, 76830, 0)]",1


Identifying the customers that they still have negative balance (still some pending payments.

In [ ]:
# This function would return the number of days that
# the balance is still in pending
def Compute_pending_days(procedure_date, lastest_date):

  # If there is a data that the balance got negative
  if procedure_date:

    # compute the differece
    return time_difference(procedure_date, lastest_date).days

  #Otherwise, return none
  return None


# Identify the latest time based on the records
Latest_time = Charges_df_copy['Encounter'].values[-1]

# Check for how many days the customer is in pending
Customers_agg_df['Pending_time'] = Customers_agg_df.apply(lambda x: Compute_pending_days(x['Negative_balance_date'], Latest_time) , axis = 1)

Customers_agg_df

,Customer id,Balance,Last_procedure_code,Last_procedure_name,Last_procedure_doctor,Negative_balance_date,Negative_balance_periods,#Negative_periods,Pending_time
0,100018,0.0,None,None,None,None,[],0,NaN
1,106033,1020.0,None,None,None,None,[],0,NaN
2,120163,-125.0,S4027Y6,Storage Embryo Monthly Over 5 Years,5,12/22/2023,[],0,21.0
3,101400,73530.54,None,None,None,None,[],0,NaN
4,106071,16318.57,None,None,None,None,[],0,NaN
...,...,...,...,...,...,...,...,...,...
5272,103593,-75.0,89343Q,Storage Sperm Quarterly,0,12/28/2023,[],0,15.0
5273,104166,0.0,None,None,None,None,[],0,NaN
5274,104232,-250.0,99205,New Patient Office Or Other Outpatient Visit L...,1,12/28/2023,[],0,15.0
5275,103935,-166.0,89320,Semen Analysis Comprehensive,5,12/28/2023,[],0,15.0


Check the number of customers that they have still pending payments.

In [ ]:
# Identifying the customers that are still in debt
In_debt_cust = Customers_agg_df[~Customers_agg_df['Pending_time'].isnull()]

In_debt_cust

,Customer id,Balance,Last_procedure_code,Last_procedure_name,Last_procedure_doctor,Negative_balance_date,Negative_balance_periods,#Negative_periods,Pending_time
2,120163,-125.0,S4027Y6,Storage Embryo Monthly Over 5 Years,5,12/22/2023,[],0,21.0
8,121227,-313.0,S4027Y35,Storage Embryo Monthly Yrs 3-5,3,10/28/2023,[],0,76.0
11,126674,-770.24,FWD,Balance Forward,0,12/14/2019,[],0,1490.0
43,119502,-125.0,S4027Y6,Storage Embryo Monthly Over 5 Years,5,12/20/2023,[],0,23.0
55,123461,-377.0,S4027Y35,Storage Embryo Monthly Yrs 3-5,9,7/7/2023,[],0,189.0
...,...,...,...,...,...,...,...,...,...
5271,103822,-16.0,99000,Specimen Handling,5,12/27/2023,[],0,16.0
5272,103593,-75.0,89343Q,Storage Sperm Quarterly,0,12/28/2023,[],0,15.0
5274,104232,-250.0,99205,New Patient Office Or Other Outpatient Visit L...,1,12/28/2023,[],0,15.0
5275,103935,-166.0,89320,Semen Analysis Comprehensive,5,12/28/2023,[],0,15.0


## Pending payments

Here is the statistics about the payments that are still pending.

In [ ]:
# We are going to consider the customers that they have still some payments to be paid
# Debt statistics
debt_stat = {'#In debt customers' : In_debt_cust.shape[0],
             'In debt total' : In_debt_cust['Balance'].abs().sum(),
             'In debt avg' : In_debt_cust['Balance'].abs().mean(),
             'In debt max' : In_debt_cust['Balance'].abs().max(),
             'In debt min' : In_debt_cust['Balance'].abs().min(),
             'In debt days avg' : In_debt_cust['Pending_time'].mean(),
             'In debt days max' : In_debt_cust['Pending_time'].max(),
             'In debt days min' : In_debt_cust['Pending_time'].min()}

pd.DataFrame([debt_stat])

,#In debt customers,In debt total,In debt avg,In debt max,In debt min,In debt days avg,In debt days max,In debt days min
0,647,898424.96,1388.601175,31394.15,0.29,90.859351,1490.0,14.0


#Delinquent customers

Regarding identifying delinquent customers we will consider two different time periods, 30 and 60 days and give some insights about those different periods.

### 30 days delinquent

First identifying the customers that they are in pending for more than 30 days

In [ ]:
# customers that they are in pending for more than 30 days
delinquent_30_cust = In_debt_cust[In_debt_cust['Pending_time'] >= 30 ]

delinquent_30_cust

,Customer id,Balance,Last_procedure_code,Last_procedure_name,Last_procedure_doctor,Negative_balance_date,Negative_balance_periods,#Negative_periods,Pending_time
8,121227,-313.0,S4027Y35,Storage Embryo Monthly Yrs 3-5,3,10/28/2023,[],0,76.0
11,126674,-770.24,FWD,Balance Forward,0,12/14/2019,[],0,1490.0
55,123461,-377.0,S4027Y35,Storage Embryo Monthly Yrs 3-5,9,7/7/2023,[],0,189.0
86,123600,-63.0,S4027Y35,Storage Embryo Monthly Yrs 3-5,6,12/8/2023,[],0,35.0
93,123764,-8294.25,76830,Ultrasound Transvaginal,3,12/3/2023,[],0,40.0
...,...,...,...,...,...,...,...,...,...
5216,123717,-42.44,99214-95,Office Visit Established Patient Level 4,9,12/12/2023,[],0,31.0
5219,104195,-885.0,99499,Unlisted Evaluation And Management Service,1,12/12/2023,[],0,31.0
5225,104207,-1196.0,99000,Specimen Handling,9,12/13/2023,[],0,30.0
5226,104198,-117.0,99205-95,New Patient Office Or Other Outpatient Visit L...,1,12/13/2023,[],0,30.0


#### Statistics


Here we are giving some statistics about the customers that they are deliquent for 30 days

In [ ]:
# Giving some statistics about delinquent customers for 30 days
delinquent_30_stat = {'#Delinquent customers' : delinquent_30_cust.shape[0],
             'Total amount' : delinquent_30_cust['Balance'].abs().sum(),
             'Amount per customer' : delinquent_30_cust['Balance'].abs().mean(),
             'Max amount' : delinquent_30_cust['Balance'].abs().max(),
             'Min amount' : delinquent_30_cust['Balance'].abs().min(),
             'Average delinquent days' : (delinquent_30_cust['Pending_time']-30).mean(),
             'Maximum delinquent days' : (delinquent_30_cust['Pending_time']-30).max(),
             'Minimum deliqneunt days' : (delinquent_30_cust['Pending_time']-30).min()}

pd.DataFrame([delinquent_30_stat])

,#Delinquent customers,Total amount,Amount per customer,Max amount,Min amount,Average delinquent days,Maximum delinquent days,Minimum deliqneunt days
0,453,858603.39,1895.371722,31394.15,0.29,91.11479,1460.0,0.0


#### Top 5 procedure

Here we are going to identify the procedure that made the balance to get negative

In [ ]:
# Get the count of each procedure
procedure_count = delinquent_30_cust['Last_procedure_name'].value_counts()[:5]

#Create a bar plot
fig = px.bar(x=procedure_count.index, y=procedure_count.values, title='Top 5 last procedure deliquent customers for 30 days', labels={'x': 'Procedure', 'y': '#Cases'},  text_auto='.2s')

# Show the plot
fig.show()

#### Demographic insights

Check the demographic data of the customers that are deliqneunt for 30 days

In [ ]:
# Get the id of the customers that are in debt
delinquent_30_cust_id = list(delinquent_30_cust['Customer id'].values)

# Filtering out the information of the people that are in debt
delinquent_30_demograph = demographic_data_df[demographic_data_df['Unique Acct #'].isin(delinquent_30_cust_id)]

delinquent_30_demograph

,Unique Acct #,Assigned Doctor,State,Sex at birth,Race,Ethnicity,Marital Status,Patient Source,Age
378,110337,5.0,CA,F,Not Set,Not Set,NaN,NaN,50.0
697,110624,5.0,CA,F,Not Set,Not Set,Married,NaN,47.0
1046,101093,9.0,CA,F,Asian,Not Hispanic or Latino,Married,Friend,42.0
1702,101152,2.0,CA,F,White,Hispanic or Latino,Single,Doctor,30.0
1768,101158,9.0,CA,F,Declined,Declined,Married,Doctor,40.0
...,...,...,...,...,...,...,...,...,...
28344,109520,5.0,CA,F,Not Set,Not Set,NaN,NaN,50.0
28704,109845,8.0,CA,F,Not Set,Not Set,NaN,NaN,46.0
28713,109853,5.0,CA,F,Not Set,Not Set,NaN,NaN,54.0
28738,109876,5.0,CA,M,Not Set,Not Set,NaN,NaN,64.0


#### Gender distribution

In [ ]:
# Get the distribution of the delinquent for 30 days custmoers
delinquent_30_gender = delinquent_30_demograph['Sex at birth'].value_counts()

#Create a bar plot
fig = px.bar(x=delinquent_30_gender.index, y=delinquent_30_gender.values, title='Delinquent customers for 30 days - Gender distribution', labels={'x': 'Gender', 'y': '#Delinquent customers '},  text_auto='.2s')

# Show the plot
fig.show()

#### Doctors insight

Checking the doctor of the delinquent customers

In [ ]:
# Check the count of the number of the delinquent customers
# for the doctors
delinquent_30_doctor = delinquent_30_demograph['Assigned Doctor'].astype('object').value_counts()

#Create a bar plot
fig = px.bar(x=list(map(str, delinquent_30_doctor.index)), y=delinquent_30_doctor.values, title='#Delinquent for 30 days customers of each doctors ', labels={'x': 'Doctor', 'y': '#Delinquent customers'},  text_auto='.2s')

# Show the plot
fig.show()

#### Age insights

In [ ]:
# Debt statistics
age_stat = {'In debt age avg' : delinquent_30_demograph['Age'].mean(),
            'In debt age max' : delinquent_30_demograph['Age'].max(),
            'In debt age min' : delinquent_30_demograph['Age'].min()}

pd.DataFrame([age_stat])

,In debt age avg,In debt age max,In debt age min
0,39.328918,67.0,16.0


#### Race insights

In [ ]:
# Check the race distribution of the delinquent for 30 days customers
delinquent_30_race = delinquent_30_demograph['Race'].value_counts()

#Create a bar plot
fig = px.bar(x=delinquent_30_race.index, y=delinquent_30_race.values, title='Race distribution of deliquent for 30 days ', labels={'x': 'Race', 'y': '#Delinquent customers'},  text_auto='.2s')

# Show the plot
fig.show()

In [ ]:
# Check the ethnicity distribution of the delinquent for 30 days customers
delinquent_30_ethnicity = delinquent_30_demograph['Ethnicity'].value_counts()

#Create a bar plot
fig = px.bar(x=delinquent_30_ethnicity.index, y=delinquent_30_ethnicity.values, title='Ethnicity distribution of deliquent for 30 days ', labels={'x': 'ethnicity', 'y': '#Delinquent customers'},  text_auto='.2s')

# Show the plot
fig.show()

### 60 days delinquent

First identifying the customers that they are in pending for more than 60 days

In [ ]:
# customers that they are in pending for more than 60 days
delinquent_60_cust = In_debt_cust[In_debt_cust['Pending_time'] >= 60 ]

delinquent_60_cust

,Customer id,Balance,Last_procedure_code,Last_procedure_name,Last_procedure_doctor,Negative_balance_date,Negative_balance_periods,#Negative_periods,Pending_time
8,121227,-313.0,S4027Y35,Storage Embryo Monthly Yrs 3-5,3,10/28/2023,[],0,76.0
11,126674,-770.24,FWD,Balance Forward,0,12/14/2019,[],0,1490.0
55,123461,-377.0,S4027Y35,Storage Embryo Monthly Yrs 3-5,9,7/7/2023,[],0,189.0
98,123758,-1030.0,89342,Storage Embryo Yearly,5,9/6/2023,[],0,128.0
101,128261,-1470.0,76700,Ultrasound Abdominal B Scan And Or Real Time With,0,10/6/2023,[],0,98.0
...,...,...,...,...,...,...,...,...,...
5040,103758,-302.0,58100,Endometrial Biopsy,9,10/31/2023,[],0,73.0
5052,123996,-152.7,99205-95,New Patient Office Or Other Outpatient Visit L...,8,11/1/2023,[],0,72.0
5055,103942,-885.0,99499,Unlisted Evaluation And Management Service,1,11/2/2023,[],0,71.0
5069,103814,-110.0,99205,New Patient Office Or Other Outpatient Visit L...,1,11/6/2023,[],0,67.0


#### Statistics


Here we are giving some statistics about the customers that they are deliquent for 30 days

In [ ]:
# Giving some statistics about delinquent customers for 60 days
delinquent_60_stat = {'#Delinquent customers' : delinquent_60_cust.shape[0],
             'Total amount' : delinquent_60_cust['Balance'].abs().sum(),
             'Amount per customer' : delinquent_60_cust['Balance'].abs().mean(),
             'Max amount' : delinquent_60_cust['Balance'].abs().max(),
             'Min amount' : delinquent_60_cust['Balance'].abs().min(),
             'Average delinquent days' : (delinquent_60_cust['Pending_time']-60).mean(),
             'Maximum delinquent days' : (delinquent_60_cust['Pending_time']-60).max(),
             'Minimum deliqneunt days' : (delinquent_60_cust['Pending_time']-60).min()}

pd.DataFrame([delinquent_60_stat])

,#Delinquent customers,Total amount,Amount per customer,Max amount,Min amount,Average delinquent days,Maximum delinquent days,Minimum deliqneunt days
0,281,483327.72,1720.027473,30262.1,0.29,108.779359,1430.0,0.0


#### Top 5 procedure

Here we are going to identify the procedure that made the balance to get negative

In [ ]:
# Get the count of each procedure
procedure_count = delinquent_60_cust['Last_procedure_name'].value_counts()[:5]

#Create a bar plot
fig = px.bar(x=procedure_count.index, y=procedure_count.values, title='Top 5 last procedure deliquent customers for 60 days', labels={'x': 'Procedure', 'y': '#Cases'},  text_auto='.2s')

# Show the plot
fig.show()

#### Demographic insights

Check the demographic data of the customers that are deliqneunt for 60 days

In [ ]:
# Get the id of the customers that are in debt
delinquent_60_cust_id = list(delinquent_60_cust['Customer id'].values)

# Filtering out the information of the people that are in debt
delinquent_60_demograph = demographic_data_df[demographic_data_df['Unique Acct #'].isin(delinquent_60_cust_id)]

delinquent_60_demograph

,Unique Acct #,Assigned Doctor,State,Sex at birth,Race,Ethnicity,Marital Status,Patient Source,Age
697,110624,5.0,CA,F,Not Set,Not Set,Married,NaN,47.0
1046,101093,9.0,CA,F,Asian,Not Hispanic or Latino,Married,Friend,42.0
1702,101152,2.0,CA,F,White,Hispanic or Latino,Single,Doctor,30.0
1924,101172,6.0,CA,F,Asian,Not Hispanic or Latino,Married,Doctor,42.0
2200,111978,5.0,CA,F,Black or African American,Not Hispanic or Latino,Married,Family Member,42.0
...,...,...,...,...,...,...,...,...,...
28058,109263,5.0,CA,F,Not Set,Not Set,NaN,NaN,59.0
28110,109309,5.0,CA,F,White,Not Hispanic or Latino,Married,Word of Mouth,64.0
28142,100933,2.0,CA,F,White,Hispanic or Latino,Single,NaN,37.0
28284,109467,5.0,CA,F,Not Set,Not Set,NaN,NaN,49.0


#### Gender distribution

In [ ]:
# Get the distribution of the delinquent for 60 days custmoers
delinquent_60_gender = delinquent_60_demograph['Sex at birth'].value_counts()

#Create a bar plot
fig = px.bar(x=delinquent_60_gender.index, y=delinquent_60_gender.values, title='Delinquent customers for 60 days - Gender distribution', labels={'x': 'Gender', 'y': '#Delinquent customers '},  text_auto='.2s')

# Show the plot
fig.show()

#### Doctors insight

Checking the doctor of the delinquent customers

In [ ]:
# Check the count of the number of the delinquent customers
# for the doctors
delinquent_60_doctor = delinquent_60_demograph['Assigned Doctor'].astype('object').value_counts()

#Create a bar plot
fig = px.bar(x=list(map(str, delinquent_60_doctor.index)), y=delinquent_60_doctor.values, title='#Delinquent for 60 days customers of each doctors ', labels={'x': 'Doctor', 'y': '#Delinquent customers'},  text_auto='.2s')

# Show the plot
fig.show()

#### Age insights

In [ ]:
# Debt statistics
age_stat = {'In debt age avg' : delinquent_60_demograph['Age'].mean(),
            'In debt age max' : delinquent_60_demograph['Age'].max(),
            'In debt age min' : delinquent_60_demograph['Age'].min()}

pd.DataFrame([age_stat])

,In debt age avg,In debt age max,In debt age min
0,38.957295,65.0,19.0


#### Race insights

In [ ]:
# Check the race distribution of the delinquent for 60 days customers
delinquent_60_race = delinquent_60_demograph['Race'].value_counts()

#Create a bar plot
fig = px.bar(x=delinquent_60_race.index, y=delinquent_60_race.values, title='Race distribution of deliquent for 60 days ', labels={'x': 'Race', 'y': '#Delinquent customers'},  text_auto='.2s')

# Show the plot
fig.show()

In [ ]:
# Check the ethnicity distribution of the delinquent for 60 days customers
delinquent_60_ethnicity = delinquent_60_demograph['Ethnicity'].value_counts()

#Create a bar plot
fig = px.bar(x=delinquent_60_ethnicity.index, y=delinquent_60_ethnicity.values, title='Ethnicity distribution of deliquent for 60 days ', labels={'x': 'ethnicity', 'y': '#Delinquent customers'},  text_auto='.2s')

# Show the plot
fig.show()